This notebook will study the convergence and "error bars" for the accuracy of $f_N^*$, the optimal function for noisy inputs, by evaluating these functions on finite datasets.

The current experiment for counterexamples trains an SAN to find the lowest error function on a training (or validation!) set, call this $\hat{f}_{train}$ or $\hat{f}_{test}$. The definition of $\hat{f}_{test}$ is that it minimizes error on a _specific_ test set, call it $D_N$ where $N$ is the number of data. This immediately gives two statistical fluctuations:
 1. For different $D_N$, we get different $\hat{f}_{test}$
 2. Any specific $\hat{f}_{test}$ will achieve different accuracies when evaluated on different $D_N$

We want to show the (error, sensitivity) point that $\hat{f}_{test}$ is _attracted towards_. What does this mean? Suppose that we have a function $h$ that we are computing (error, sens) for.
 - this coordinate should be(?) specific for the dataset: $\hat{f}$ cannot "see" alternative datasets.
 - This coordinate could therefore be the error of the $h$ that achieves optimal performance on $D_N$
 - This coordinate _might not_ be the same as the coordinate for $f_N^*$
 - A plausible choice for $h$ should be the optimal function for the specific dataset Matheus is looking at

counterexample = "00100110"